In [1]:
import mxnet as mx
from mxnet import gluon, autograd, ndarray
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

In [25]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

### Normalize data for the NN

In [26]:
df_norm = (df - df.mean()) / (df.max() - df.min())

### Split the data

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df_norm, y,
                                                    test_size=0.2, random_state=1111)

### Gluon Model

In [28]:
BATCH_SIZE = 32
LEARNING_R = 0.001
EPOCHS = 150

### Prepare data 

In [30]:
train_dataset = mx.gluon.data.ArrayDataset(X_train.as_matrix(),y_train)
test_dataset = mx.gluon.data.ArrayDataset(X_test.as_matrix(),y_test)


In [31]:
train_data = mx.gluon.data.DataLoader(train_dataset,
                                      batch_size=BATCH_SIZE, shuffle=True)

test_data = mx.gluon.data.DataLoader(test_dataset,
                                     batch_size=BATCH_SIZE, shuffle=False)

### Initialize the model

In [32]:
net = gluon.nn.Sequential()

# Define the model architecture
with net.name_scope():
    net.add(gluon.nn.Dense(64, activation="relu"))
    #net.add(gluon.nn.Dropout(0.8)) 
    net.add(gluon.nn.Dense(32, activation="relu") ) 
    net.add(gluon.nn.BatchNorm())    
    net.add(gluon.nn.Dense(1,  activation="sigmoid"))

# Intitalize parametes of the model
net.collect_params().initialize(mx.init.Uniform())

# Add binary loss function
binary_cross_entropy = gluon.loss.SigmoidBinaryCrossEntropyLoss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': LEARNING_R})


In [33]:
for e in range(EPOCHS):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(mx.cpu()).astype('float32')
        label = label.as_in_context(mx.cpu()).astype('float32')
        with autograd.record(): # Start recording the derivatives
            output = net(data) # the forward iteration
            loss = binary_cross_entropy(output, label)
            loss.backward()
        trainer.step(data.shape[0])
        # Provide stats on the improvement of the model over each epoch
        curr_loss = ndarray.mean(loss).asscalar()
    if e % 20 == 0:
        print("Epoch {}. Current Loss: {}.".format(e, curr_loss))

Epoch 0. Current Loss: 0.613682210445404.
Epoch 20. Current Loss: 0.5531467199325562.
Epoch 40. Current Loss: 0.5215882658958435.
Epoch 60. Current Loss: 0.5606438517570496.
Epoch 80. Current Loss: 0.49790647625923157.
Epoch 100. Current Loss: 0.4782971739768982.
Epoch 120. Current Loss: 0.4901566505432129.
Epoch 140. Current Loss: 0.5007529854774475.


### Predict

In [34]:
y_pred = np.array([])
for data,label in test_data:
        data = data.as_in_context(mx.cpu()).astype('float32')
        label = label.as_in_context(mx.cpu()).astype('float32')
        output = net(data)
        y_pred = np.append(y_pred, output.asnumpy())

In [35]:
y_pred_labels = np.where(y_pred > 0.48, 1, 0)

In [36]:
accuracy_score(y_test, y_pred_labels)

0.98245614035087714